In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('JC-202504-citibike-tripdata.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81553 entries, 0 to 81552
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ride_id             81553 non-null  object 
 1   rideable_type       81553 non-null  object 
 2   started_at          81553 non-null  object 
 3   ended_at            81553 non-null  object 
 4   start_station_name  81553 non-null  object 
 5   start_station_id    81553 non-null  object 
 6   end_station_name    81329 non-null  object 
 7   end_station_id      81292 non-null  object 
 8   start_lat           81553 non-null  float64
 9   start_lng           81553 non-null  float64
 10  end_lat             81531 non-null  float64
 11  end_lng             81531 non-null  float64
 12  member_casual       81553 non-null  object 
dtypes: float64(4), object(9)
memory usage: 8.1+ MB


In [16]:
df.isnull().sum()

ride_id                 0
rideable_type           0
started_at              0
ended_at                0
start_station_name      0
start_station_id        0
end_station_name      224
end_station_id        261
start_lat               0
start_lng               0
end_lat                22
end_lng                22
member_casual           0
dtype: int64

In [17]:
df['end_station_name'] = df['end_station_name'].fillna('Unknown')
df['end_station_id'] = df['end_station_id'].fillna('Unknown')

In [18]:
avg_coords = df.groupby('end_station_name')[['end_lat','end_lng']].mean()

In [19]:
#calculate average coordinates per station
avg_coords = df.groupby('end_station_name')[['end_lat', 'end_lng']].mean()

#map averages dataframe and fill in missing values
df['avg_lat'] = df['end_station_name'].map(avg_coords['end_lat'])
df['avg_lng'] = df['end_station_name'].map(avg_coords['end_lng'])

df['end_lat'] = df['end_lat'].fillna(df['avg_lat'])
df['end_lng'] = df['end_lng'].fillna(df['avg_lng'])

df.drop(columns=['avg_lat', 'avg_lng'], inplace=True)

In [20]:
print(df.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')


In [21]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['ride_duration'] = (df['ended_at']- df['started_at']).dt.total_seconds()/60
df['day_of_week'] = df['started_at'].dt.day_name()
df['hour'] = df['started_at'].dt.month
df['loop_ride'] = df['start_station_name'] == df['end_station_name']



In [22]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration,day_of_week,hour,loop_ride
0,6B10405DA9D9A760,electric_bike,2025-04-24 22:29:40.611,2025-04-24 22:31:48.180,Clinton St & Newark St,HB409,Clinton St & Newark St,HB409,40.73743,-74.03571,40.737430,-74.035710,member,2.126150,Thursday,4,True
1,E0FFCF6AAA7EFD5E,classic_bike,2025-04-10 18:07:39.559,2025-04-10 18:12:16.533,Clinton St & Newark St,HB409,Mama Johnson Field - 4 St & Jackson St,HB404,40.73743,-74.03571,40.743140,-74.040041,member,4.616233,Thursday,4,False
2,6CF18FF246216C41,classic_bike,2025-04-16 22:54:04.131,2025-04-16 23:00:21.703,Clinton St & Newark St,HB409,9 St HBLR - Jackson St & 8 St,HB305,40.73743,-74.03571,40.747907,-74.038412,member,6.292867,Wednesday,4,False
3,3761CCD0D1489C70,classic_bike,2025-04-17 15:38:03.338,2025-04-17 15:47:54.587,Clinton St & Newark St,HB409,14 St Ferry - 14 St & Shipyard Ln,HB202,40.73743,-74.03571,40.752961,-74.024353,casual,9.854150,Thursday,4,False
4,E6D847415B5A22DF,classic_bike,2025-04-22 06:46:53.286,2025-04-22 06:49:10.829,Clinton St & Newark St,HB409,River St & 1 St,HB609,40.73743,-74.03571,40.737215,-74.028865,member,2.292383,Tuesday,4,False


In [23]:
df.to_csv('cleaned_bikeshare_data.csv', index=False)